In [ ]:
# Libraries
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np

### Helper Functions

In [ ]:
# Takes YouTube's video categories from json files and creates a DF with category names
def convert_json_cats_to_pandas(json_dir):
    # Load in the categories
    with open(json_dir) as f:
        cat_data = json.load(f)

    cat = {'category_id': [], 'cat_name': []}

    for i in cat_data['items']:
        cat['category_id'].append(i['id'])
        cat['cat_name'].append(i['snippet']['title'])

    df = pd.DataFrame(data=cat)

    return df

In [ ]:
# Transforms and cleans raw data. Returns a DF with likes to views ratio
def get_merged_data(vid_path, cat_path, year, country_code):
    # Import the video trending data
    vids_df = pd.read_csv(vid_path)
    
    # Add country to the dataframe
    vids_df['country'] = country_code
    
    # Clean 2020 data
    if year == 2020:
        vids_df['category_id'] = vids_df['categoryId']
        vids_df['views'] = vids_df['view_count']
        vids_df['publish_time'] = vids_df['publishedAt']
        vids_df['channel_title'] = vids_df['channelTitle']

    cats_df = convert_json_cats_to_pandas(cat_path)
    cats_df['category_id'] = cats_df['category_id'].astype(int)
    
    # Merge video and category dataframes
    df = pd.merge(vids_df, cats_df, on='category_id')
    
    # Clean 2018 data and filter first 4 months
    if year == 2018:
        df['trending_date'] = pd.to_datetime(df['trending_date'], format='%y.%d.%m')
        df['trending_year'] = df['trending_date'].dt.year
        df['trending_month'] = df['trending_date'].dt.month
        df = df[df['trending_year'] == 2018]
        df = df[df['trending_month'] <= 4]
    
    # Select only columns we are interested in
    likes_views_df = df[['title', 'channel_title', 'category_id', 'cat_name', 'views',
                                    'likes', 'dislikes', 'trending_date', 'country']].copy(deep=True)
    
    likes_views_df = likes_views_df[likes_views_df['cat_name'] != 'Shows']
    likes_views_df = likes_views_df[likes_views_df['cat_name'] != 'Movies']
    
    # Create column of likes to views ratio
    likes_views_df['likes_to_views'] = (likes_views_df['likes'] / likes_views_df['views'])
        
    return likes_views_df

### Extract-Transform YouTube Data

#### Extract-Transform, Save 2018 Data

In [ ]:
# Get 2018 data for the year and each country
countries = ['CA', 'GB', 'US']

files_df = pd.DataFrame({'vid_paths': ['./2018 Data/CAvideos.csv.zip', './2018 Data/GBvideos.csv.zip', './2018 Data/USvideos.csv.zip'],
                         'cat_paths': ['./2018 Data/CA_category_id.json','./2018 Data/GB_category_id.json','./2018 Data/US_category_id.json'],
                         'countries': ['CA', 'GB', 'US']
                        })

column_list = {'title', 'channel_title', 'category_id', 'cat_name', 'views','likes', 'dislikes', 'trending_date', 'country'}

# merge dataframes together
total_2018_df = pd.DataFrame()
for i, file in files_df.iterrows():
    df = get_merged_data(file['vid_paths'], file['cat_paths'], 2018, file['countries'])
    df.to_csv('./transformed-data/2018_' + file['countries'] + '_data.csv')
    print(file['countries'], df.shape[0])
    
    # append data for yearly total
    total_2018_df = total_2018_df.append(df, ignore_index=True)

total_2018_df.to_csv('./transformed-data/data_2018_all.csv')
print("Total trending videos in 2018 (Jan - Apr): " + str(total_2018_df.shape[0]))

#### Extract-Transform, Save 2020 Data

In [ ]:
# Get 2020 data for the year and each country
countries = ['CA', 'GB', 'US']

files_df = pd.DataFrame({'vid_paths': ['./2020 Data/CA_youtube_trending_data.csv.zip', './2020 Data/GB_youtube_trending_data.csv.zip', './2020 Data/US_youtube_trending_data.csv.zip'],
                         'cat_paths': ['./2020 Data/CA_category_id.json','./2020 Data/GB_category_id.json','./2020 Data/US_category_id.json'],
                         'countries': ['CA', 'GB', 'US']
                        })

column_list = {'title', 'channel_title', 'category_id', 'cat_name', 'views','likes', 'dislikes', 'trending_date', 'country'}

# merge dataframes together
total_2020_df = pd.DataFrame()
for i, file in files_df.iterrows():
    df = get_merged_data(file['vid_paths'], file['cat_paths'], 2020, file['countries'])
    df.to_csv('./transformed-data/2020_' + file['countries'] + '_data.csv')
    print(file['countries'], df.shape[0])
    
    # append data for yearly total
    total_2020_df = total_2020_df.append(df, ignore_index=True)

total_2020_df.to_csv('./transformed-data/data_2020_all.csv')
print("Total trending videos in 2020 (Aug - Oct): " + str(total_2020_df.shape[0]))